<a href="https://colab.research.google.com/github/Mohan5353/DS_ML_DL/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from pprint import pprint

In [2]:
ds,info=tfds.load("tf_flowers",as_supervised=True,with_info=True)

In [3]:
pprint(info)

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_dir='/root/tensorflow_datasets/tf_flowers/3.0.1',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)


In [4]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info. features["label"].num_classes

In [5]:
test_ds, valid_ds, train_ds = tfds.load("tf_flowers", split=["train[:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True)

In [6]:
def preprocess(image,label):
    resized_image=tf.image.resize(image,[224,224])
    result=tf.keras.applications.xception.preprocess_input(resized_image)
    return result,label

In [7]:
batch_size = 32
train_ds = train_ds.shuffle(1000)
train_ds = train_ds.map(preprocess).batch(batch_size).prefetch(1)
valid_ds = valid_ds.map(preprocess).batch(batch_size).prefetch(1)
test_ds = test_ds.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
base_model=tf.keras.applications.xception.Xception(weights="imagenet",include_top=False)
avg_layer=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output=tf.keras.layers.Dense(n_classes,activation="softmax")(avg_layer)
model=tf.keras.Model(inputs=base_model.input,outputs=output)

In [9]:
for layer in base_model.layers:
  layer.training=False

In [10]:
optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.05,momentum=0.9,decay=0.01)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy","mse"])
history=model.fit(train_ds,epochs=5,validation_data=valid_ds)

Epoch 1/5
86/86 [==============================] - 59s 507ms/step - loss: 0.5946 - accuracy: 0.7965 - mse: 4.9520 - val_loss: 4.6500 - val_accuracy: 0.5009 - val_mse: 4.9513
Epoch 2/5
86/86 [==============================] - 42s 485ms/step - loss: 0.2452 - accuracy: 0.9117 - mse: 4.9852 - val_loss: 0.9534 - val_accuracy: 0.7858 - val_mse: 4.9519
Epoch 3/5
86/86 [==============================] - 42s 486ms/step - loss: 0.1125 - accuracy: 0.9677 - mse: 4.9972 - val_loss: 0.2480 - val_accuracy: 0.9074 - val_mse: 4.9492
Epoch 4/5
86/86 [==============================] - 42s 488ms/step - loss: 0.0475 - accuracy: 0.9840 - mse: 5.0027 - val_loss: 0.2574 - val_accuracy: 0.9129 - val_mse: 4.9528
Epoch 5/5
86/86 [==============================] - 42s 487ms/step - loss: 0.0161 - accuracy: 0.9964 - mse: 5.0060 - val_loss: 0.2458 - val_accuracy: 0.9310 - val_mse: 4.9572


In [11]:
for layer in base_model.layers:
  layer.training=True

In [ ]:
optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.01,momentum=0.9,decay=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy","mse"])
history=model.fit(train_ds,epochs=20,validation_data=valid_ds)

Epoch 1/20
20/86 [=====>........................] - ETA: 29s - loss: 0.0063 - accuracy: 0.9984 - mse: 4.9108